# Métrica 2_4_01 — Desviaciones en el ingreso según nivel jerárquico

| Escenario                                                      | Condición      | Resultado        |
| -------------------------------------------------------------- | -------------- | ---------------- |
| Falta de datos o no obligatorio                                | —              | ⚪ **SIN_DATO**   |
| Ingreso dentro del rango intercuartílico (Q1–Q3) para su nivel | ✅              | 🟢 **CUMPLE**    |
| Ingreso fuera de ese rango                                     | ❌              | 🔴 **NO_CUMPLE** |
| Sin referencia para ese nivel (pocos datos en el grupo)        | ⚪ **SIN_DATO** |                  |


In [1]:
import traceback
import numpy as np
import pandas as pd
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "2_4_01_DESVIACION_INGRESO_NIVEL"

DICT_INFO = {
    "nivel_path": "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelEmpleoCargoComision",
    "ingreso_path": "declaracion.situacionPatrimonial.ingresos.ingresoMensualNetoDeclarante.valor",
    "obligatorio": False
}

# --- Funciones auxiliares ---

def obtener_valor(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p, {})
            else:
                return None
        if isinstance(actual, (int, float)):
            return actual
        try:
            return float(actual)
        except Exception:
            return None
    except Exception:
        return None

def obtener_texto(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p, {})
            else:
                return None
        if isinstance(actual, dict):
            return actual.get("valor") or actual.get("clave") or None
        return str(actual).strip() if actual else None
    except Exception:
        return None

# --- Construcción de referencia (IQR por nivel) ---

def construir_referencia(source):
    niveles = []
    for d in source.find({}, {
        "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelEmpleoCargoComision": 1,
        "declaracion.situacionPatrimonial.ingresos.ingresoMensualNetoDeclarante.valor": 1
    }):
        nivel = obtener_texto(d, DICT_INFO["nivel_path"])
        ingreso = obtener_valor(d, DICT_INFO["ingreso_path"])
        if nivel and isinstance(ingreso, (int, float)) and ingreso > 0:
            niveles.append({"nivel": nivel, "ingreso": ingreso})

    df = pd.DataFrame(niveles)
    if df.empty:
        return {}

    referencia = {}
    for nivel, grupo in df.groupby("nivel"):
        if len(grupo) < 5:
            continue
        q1 = grupo["ingreso"].quantile(0.25)
        q3 = grupo["ingreso"].quantile(0.75)
        referencia[nivel] = (q1, q3)
    return referencia

# --- Evaluación de la métrica ---

def evaluar_metrica(doc, referencia):
    nivel = obtener_texto(doc, DICT_INFO["nivel_path"])
    ingreso = obtener_valor(doc, DICT_INFO["ingreso_path"])

    if nivel is None or ingreso is None:
        return "SIN_DATO"

    if nivel not in referencia:
        return "SIN_DATO"

    q1, q3 = referencia[nivel]
    if q1 <= ingreso <= q3:
        return "CUMPLE"
    return "NO_CUMPLE"

# --- Procesamiento MongoDB ---

def procesar_metrica_2_4_01():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0}
    operaciones = []
    total = 0
    client = None

    try:
        print(f"Procesando métrica {METRIC_ID}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        # Construir tabla de referencia IQR por nivel jerárquico
        print("Construyendo referencia de ingresos por nivel jerárquico...")
        referencia = construir_referencia(src)
        print(f"Niveles de referencia generados: {len(referencia)}")

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelEmpleoCargoComision": 1,
            "declaracion.situacionPatrimonial.ingresos.ingresoMensualNetoDeclarante.valor": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1
            try:
                resultado = evaluar_metrica(doc, referencia)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error en doc {doc.get('_id')}: {e}")

            resultados[resultado] = resultados.get(resultado, 0) + 1
            operaciones.append(UpdateOne({"_id": doc["_id"]}, {"$set": {METRIC_ID: resultado}}, upsert=True))

            if len(operaciones) >= 1000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f"  > Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}")
        for k, v in resultados.items():
            print(f"  > {k}: {v}")

    except Exception:
        traceback.print_exc()
    finally:
        if client:
            client.close()
            print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_2_4_01()


Procesando métrica 2_4_01_DESVIACION_INGRESO_NIVEL...
Construyendo referencia de ingresos por nivel jerárquico...
Niveles de referencia generados: 9301


c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


  > Procesados 1000 documentos...
  > Procesados 2000 documentos...
  > Procesados 3000 documentos...
  > Procesados 4000 documentos...
  > Procesados 5000 documentos...
  > Procesados 6000 documentos...
  > Procesados 7000 documentos...
  > Procesados 8000 documentos...
  > Procesados 9000 documentos...
  > Procesados 10000 documentos...
  > Procesados 11000 documentos...
  > Procesados 12000 documentos...
  > Procesados 13000 documentos...
  > Procesados 14000 documentos...
  > Procesados 15000 documentos...
  > Procesados 16000 documentos...
  > Procesados 17000 documentos...
  > Procesados 18000 documentos...
  > Procesados 19000 documentos...
  > Procesados 20000 documentos...
  > Procesados 21000 documentos...
  > Procesados 22000 documentos...
  > Procesados 23000 documentos...
  > Procesados 24000 documentos...
  > Procesados 25000 documentos...
  > Procesados 26000 documentos...
  > Procesados 27000 documentos...
  > Procesados 28000 documentos...
  > Procesados 29000 document

KeyboardInterrupt: 